In [4]:
from nytimesarticle import articleAPI
api = articleAPI('2462ecddebb043db8fce29ca0d9b9b66')
from IPython.core.debugger import Tracer
import time
import pandas as pd
from stop_words import get_stop_words










In [5]:
#Function to parse the results of the API query into a list of dictionaries where each dictionary represents a different article


def parser(articles):
    news = []
    for article in articles['response']['docs']:
        dic = {}
        dic['date'] = article['pub_date'][0:10]
        dic['headline'] = article['headline']['main']
        news.append(dic)
    return news 

    

    
        





In [6]:
# This function performs a search of the NYTimes api for a page range and then returns the articles as a list of dictionaries 
def get_articles():
    headlines = []
    for i in range(0,10):
        article = api.search(begin_date = 20160101, end_date = 20170212,sort = 'newest',page = str(i), fl = ['headline','pub_date'])
        try:
            article = parser(article)
            headlines += article
            time.sleep(1)
        except KeyError:
            print article
    return headlines



        



In [14]:

headlines = get_articles()
headlines_df = pd.DataFrame(headlines)
headlines_df['Source'] = 'New York Times'

#General properites of the titles like min, max, and mean title length 

def headline_properties(headlines_df):
    lengths = []
    for headline in headlines_df['headline']:
        length = len(headline.split())
        lengths.append(length)
    headlines_df['title length'] = lengths

    mean_title_length = headlines_df['title length'].mean()
    max_title_length = headlines_df['title length'].max()
    min_title_length = headlines_df['title length'].min()
    
    print 'The mean title length is ' + str(mean_title_length)
    print 'The max title length is ' + str(max_title_length)
    print 'The min title length is ' + str(min_title_length)

    











    


,date,headline,Source
0,2017-02-12,G.O.P. Lawmakers Like What They See in Trump. ...,New York Times
1,2017-02-12,Ball Scores 22 to Lead No. 10 UCLA Past Oregon...,New York Times
2,2017-02-12,"Yen Slips After Trump-Abe Meet, Asian Shares Firm",New York Times
3,2017-02-12,"Turmoil at the National Security Council, From...",New York Times
4,2017-02-12,Golf Capsules,New York Times
5,2017-02-12,Asian Tech Titans Take a Page From Trump’s Book,New York Times
6,2017-02-12,Trump Responds to North Korean Missile Launch ...,New York Times
7,2017-02-12,Verizon Reintroduces Unlimited Data Plan as Co...,New York Times
8,2017-02-12,The Grammys Red Carpet 2017,New York Times
9,2017-02-12,The Grammys Red Carpet 2017,New York Times


In [20]:
tot_words = []
for headline in headlines_df['headline']:
    words = headline.split()
    for word in words:
        tot_words.append(word)

freq_dic = {}
stop_words = get_stop_words('en')

for word in tot_words:
    if (word not in freq_dic) and (word not in stop_words):
        freq_dic[word] = 1
    elif word in freq_dic:
        freq_dic[word] = freq_dic[word] + 1

freq_df = pd.DataFrame({"word": freq_dic.keys(), "frequency": freq_dic.values()})
freq_df = freq_df.sort(columns='frequency',ascending = False)

    


    

C:\Users\firstname.lastname\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [22]:
samples = headlines_df.head(10)

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=0,stop_words='english')
vectorizer.fit(samples['headline'].values)
X = vectorizer.transform(samples['headline'].values)
y = samples['Source'].values



array(['New York Times', 'New York Times', 'New York Times',
       'New York Times', 'New York Times', 'New York Times',
       'New York Times', 'New York Times', 'New York Times',
       'New York Times'], dtype=object)

In [23]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X,y)











MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)